# Fermion-Qubit transformation using OpenFermion

In [7]:
import openfermion
import numpy as np
from openfermion.ops import *
from openfermion.transforms import *
from openfermion.utils import *

def get_s2_operator(nmo):
    # generate S^2 Fermionic operator in the direct mapping
    """
    S(i,j)^2 = S_z(i)*S_z(j) + (S_+(i) * S_-(j) + S_-(i) * S_+(j))/2
    """
    s2_operator = FermionOperator()
     
    for imo in range(nmo):
        ia = 2 * imo
        ib  = 2 * imo + 1
        for jmo in range(nmo):
            ja = 2 * jmo
            jb  = 2 * jmo + 1
            # S_z(i) * S_z(j) terms
            s2_operator +=  0.25 * FermionOperator(((ia, 1), (ia, 0), (ja, 1), (ja, 0)))
            s2_operator += -0.25 * FermionOperator(((ia, 1), (ia, 0), (jb, 1), (jb, 0)))
            s2_operator += -0.25 * FermionOperator(((ib, 1), (ib, 0), (ja, 1), (ja, 0)))
            s2_operator +=  0.25 * FermionOperator(((ib, 1), (ib, 0), (jb, 1), (jb, 0)))
            # (S_+(i) * S_-(j) + S_-(i) * S_+(j))/2 terms
            s2_operator +=  0.50 * FermionOperator(((ia, 1), (ib, 0), (jb, 1), (ja, 0)))
            s2_operator +=  0.50 * FermionOperator(((ib, 1), (ia, 0), (ja, 1), (jb, 0)))    

    return s2_operator

def print_wave_function(wave_function):
    wfdim = len(wave_function)
    print("Coef  Determinant")
    for idet in range(wfdim):
        det = wave_function[idet].real
        if det != 0:
            print(det, '  {:4b}'.format(idet))

nmo = 2

s2_operator = get_s2_operator(nmo)  # Get S^2 operator using the function defined above

print("S^2 operator in the second quantized form")
print(s2_operator)

s2_jwt = jordan_wigner(s2_operator) # Apply Jordan-Wigner transformation
s2_jwt.compress()                   # S^2 operator is real, and thus delete zero imaginary
print("\n S^2 operator in JWT")
print(s2_jwt)

s2_bkt = bravyi_kitaev(s2_operator) # Apply Bravyi-Kitaev transformation
s2_bkt.compress()                   # Delete zero imaginary
print("\n S^2 operator in BKT")
print(s2_bkt)

# Generate |1100> state, (RHF configuration of e.g., H2 molecule)
rhf_state = np.zeros(16, dtype=np.complex64)
rhf_state[12] = 1.0    # 1*2^3 + 1*2^2 + 0*2^1 + 0*2^0 = 8 + 4 = 12
# Generate |1010> state, (ROHF configuration of e.g., H2 molecule)
rohf_state = np.zeros(16, dtype=np.complex64)
rohf_state[10] = 1.0   # 1*2^3 + 0*2^2 + 1*2^1 + 0*2^0 = 8 + 2 = 10

# Generate sparse operator and calculate expectation value
s2_jwt_sparse = jordan_wigner_sparse(s2_operator)
s2_value_rhf  = expectation(s2_jwt_sparse, rhf_state).real   # Expectation value calculation using the function in OpenFermion
s2_value_rohf = expectation(s2_jwt_sparse, rohf_state).real


print("\n == RHF wave function for (2e,2o) active space ==")
print_wave_function(rhf_state)
print(" <S**2(RHF)>  = {:.4f}".format(s2_value_rhf))
print("\n == ROHF wave function for (2e,2o) active space ==")
print_wave_function(rohf_state)
print(" <S**2(ROHF)> = {:.4f}".format(s2_value_rohf))

S^2 operator in the second quantized form
0.25 [0^ 0 0^ 0] +
-0.25 [0^ 0 1^ 1] +
0.25 [0^ 0 2^ 2] +
-0.25 [0^ 0 3^ 3] +
0.5 [0^ 1 1^ 0] +
0.5 [0^ 1 3^ 2] +
0.5 [1^ 0 0^ 1] +
0.5 [1^ 0 2^ 3] +
-0.25 [1^ 1 0^ 0] +
0.25 [1^ 1 1^ 1] +
-0.25 [1^ 1 2^ 2] +
0.25 [1^ 1 3^ 3] +
0.25 [2^ 2 0^ 0] +
-0.25 [2^ 2 1^ 1] +
0.25 [2^ 2 2^ 2] +
-0.25 [2^ 2 3^ 3] +
0.5 [2^ 3 1^ 0] +
0.5 [2^ 3 3^ 2] +
0.5 [3^ 2 0^ 1] +
0.5 [3^ 2 2^ 3] +
-0.25 [3^ 3 0^ 0] +
0.25 [3^ 3 1^ 1] +
-0.25 [3^ 3 2^ 2] +
0.25 [3^ 3 3^ 3]

 S^2 operator in JWT
0.75 [] +
0.125 [X0 X1 X2 X3] +
0.125 [X0 X1 Y2 Y3] +
0.125 [X0 Y1 X2 Y3] +
-0.125 [X0 Y1 Y2 X3] +
-0.125 [Y0 X1 X2 Y3] +
0.125 [Y0 X1 Y2 X3] +
0.125 [Y0 Y1 X2 X3] +
0.125 [Y0 Y1 Y2 Y3] +
-0.375 [Z0 Z1] +
0.125 [Z0 Z2] +
-0.125 [Z0 Z3] +
-0.125 [Z1 Z2] +
0.125 [Z1 Z3] +
-0.375 [Z2 Z3]

 S^2 operator in BKT
0.75 [] +
-0.125 [X0 Z1 X2] +
-0.125 [X0 Z1 X2 Z3] +
0.125 [X0 X2] +
0.125 [X0 X2 Z3] +
-0.125 [Y0 Z1 Y2] +
-0.125 [Y0 Z1 Y2 Z3] +
0.125 [Y0 Y2] +
0.125 [Y0 Y2 Z3] +
-0.125 [